In [1]:
!pip install -U spacy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 72.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import numpy as np
import spacy
import re
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.6 MB/s eta 0:00:00


In [3]:
df_product_info = pd.read_csv('product_info.csv')
KNOWN_BRANDS = df_product_info['brand_name'].unique().tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'product_info.csv'

In [4]:
CATEGORIES = list(pd.unique(df_product_info[['primary_category','secondary_category','tertiary_category']].values.ravel()))

In [5]:
import ast
df_product_info['highlights'] = df_product_info['highlights'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

In [6]:
unique_highlights = set()

for highlight_list in df_product_info['highlights']:
    if isinstance(highlight_list, list):  # Vérifier que c'est bien une liste après conversion
        unique_highlights.update(highlight_list)

HIGHLIGHTS = list(unique_highlights)

In [7]:
import pandas as pd
import numpy as np
import spacy
import re
from unidecode import unidecode

# Charger le modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")
def load_data(files):
    """Load and merge multiple CSV files into a single DataFrame, selecting only specific columns."""
    df_list = []
    loaded_files = []  # Track successfully loaded files
    columns_to_load = [
        "is_recommended", "skin_tone", "eye_color", "skin_type", "hair_color",
        "product_id", "product_name", "brand_name", "price_usd"
    ]

    for file in files:
        try:
            # Attempt to read each file, loading only the specified columns
            df = pd.read_csv(
                file,
                usecols=columns_to_load,  # Load only the specified columns
                low_memory=False,  # Avoid memory-related warnings
                encoding='latin1',  # Try alternative encoding if UTF-8 fails
                on_bad_lines='skip',  # Skip problematic rows
            )
            df_list.append(df)
            loaded_files.append(file)  # Add the file to the list of loaded files
        except pd.errors.ParserError as e:
            print(f"ParserError in file {file}: {e}")
        except UnicodeDecodeError as e:
            print(f"UnicodeDecodeError in file {file}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred with file {file}: {e}")

    # Concatenate all dataframes if no errors
    if df_list:
        print("Successfully loaded files:", loaded_files)  # Print the list of loaded files
        return pd.concat(df_list, ignore_index=True)
    else:
        print("No valid data found.")
        return None
# Configuration des catégories et marques connues
CATEGORY_MAP = {
    'rouge à lèvres': ('Makeup', 'Lip', 'Lipstick'),
    'fond de teint': ('Makeup', 'Face', 'Foundation'),
    'mascara': ('Makeup', 'Eye', 'Mascara'),
    'parfum': ('Fragrance', 'Women', 'Perfume', 'Cologne'),
    'soin': ('Skincare', 'Treatments', None),
    'shampooing': ('Hair', 'Shampoo & Conditioner', 'Shampoo',
                   'Hair Styling & Treatments', 'Hair Masks'),
    'cadeau': ('Value & Gift Sets', 'Perfume Gift Sets'),
    'gel douche': ('Bath & Body'),
    'ambiance': ('Candles & Home Scents', 'Diffusers', 'Candles'),
    'déodorant': ('Deodorant & Antiperspirant'),
    'highlighter': ('Highlighter'),
    'blush': ('Blush'),
    'eyeliner': ('Eyeliner'),
    'vernis': ('Nail')
}

def normalize_text(text):
    return unidecode(text.lower().strip())

def extract_features(query):
    doc = nlp(normalize_text(query))


# Initialisation des caractéristiques
    features = {
        'brands': [],
        'categories': {'primary': set(), 'secondary': set(), 'tertiary': set()},
        'attributes': {'skin_tone': set(), 'hair_color': set(), 'eye_color': set()},
        'price_range': None,
        'highlights_list': []
    }

    # Extraction du prix
    price_match = re.search(r"\b(under|below|moins de|moins que|pas plus de|max|maximum|au plus)\s*(\d+)[\$\€\'euros'\'dollars']?", query, re.IGNORECASE)
    if price_match:
        features['price_range'] = int(price_match.group(2))

    for brand in KNOWN_BRANDS:
        if brand.lower() in query.lower():
            features['brands'].append(brand)

    """for ent in doc.ents:
        if ent.label_ in ['ORG', 'MISC'] and ent.text.title() in KNOWN_BRANDS:
            features['brands'].append(ent.text.title())"""

    for highlight in HIGHLIGHTS:
        if highlight.lower() in query.lower():
            features['highlights_list'].append(highlight)

    for token in doc:
        for keyword, categories in CATEGORY_MAP.items():
            if keyword in token.text:
                features['categories']['primary'].add(categories[0])
                if categories[1]: features['categories']['secondary'].add(categories[1])
                if categories[2]: features['categories']['tertiary'].add(categories[2])

    return features

def apply_filters(df, features):
    filtered = df.copy()
    for level in ['primary', 'secondary', 'tertiary']:
        col = f'{level}_category'
        if features['categories'][level]:
            filtered = filtered[filtered[col].isin(features['categories'][level])]

    if features['brands']:
        filtered = filtered[filtered['brand_name'].isin(features['brands'])]

    if features['highlights_list']:
        filtered = filtered[filtered['highlights'].apply(lambda x: any(item in x for item in features['highlights_list']))]
        # Price filters
    if features['price_range'] is not None:
        filtered = filtered[filtered['price_usd'] <= features['price_range']]

    return filtered

def recommend_products(query, df_products, top_n=10):
    features = extract_features(query)
    filtered = apply_filters(df_products, features)
    if not filtered.empty:
        recommendations = (
            filtered.groupby(['product_id', 'product_name', 'brand_name', 'price_usd'])
            .agg(
                recommendation_score=('rating', 'mean'),
                match_count=('product_id', 'count')
            )
            .reset_index()
            .sort_values(['recommendation_score', 'match_count'], ascending=[False, False])
            .head(top_n)
        )
        return recommendations[['product_name', 'brand_name', 'price_usd', 'recommendation_score']]
    return pd.DataFrame()



In [8]:
query_test = 'je veux un vegan'
print(extract_features(query_test))

In [9]:
!pip install langchain_openai
!pip install langchain_core

In [ ]:
# chatbot2.py - Chatbot amélioré avec recommandations de produits

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
#from recommendations import *

# Classe ChatSession
class ChatSession:
    def __init__(self, memsize, llm, df):
        self.llm = llm
        self.memsize = memsize
        self.history = []
        self.df_products = df_products
        self.df_reviews = df_reviews

    def chat(self, prompt):
        recommendations = recommend_products(prompt, self.df_products)
        if not recommendations.empty:
            return "Voici quelques recommandations :\n" + recommendations.to_string(index=False)
        return "Désolé, je ne peux pas répondre à cette requête."

# Chargement des produits et initialisation du chatbot
df_products = pd.read_csv("product_info.csv")

# File list
files = [
    "reviews_0-250.csv",
    "reviews_250-500.csv",
    "reviews_500-750.csv",
    "reviews_750-1250.csv",
    "reviews_1250-end.csv"
]
df_reviews = load_data(files)
llm = ChatOpenAI(openai_api_key="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", model_name="tokyotech-llm/llama-3.1-swallow-70b-instruct-v0.3")
session = ChatSession(500, llm, df_products)

while (prompt := input("USER> ")) not in ["bye", "exit"]:
    print("CHATBOT> ", session.chat(prompt))
# chatbot2.py - Chatbot amélioré avec recommandations de produits

